In this notebook, I will check the input to chromopainter/finestructure, as well as create some supporting files. 

In [1]:
%run ../scripts/notebook_settings.py

In [3]:
meta_data_samples = pd.read_table("../data/metadata_with_x_missing.txt", sep=" ")
vcf = "/faststorage/project/baboondiversity/data/haploidified_chrX_males/hap_chrX_males.vcf.gz"

In [4]:
callset = allel.read_vcf(vcf)

In [9]:
individuals = callset["samples"]

In [21]:
pop_list = []
inclusion_list = []
for i in individuals:
    if meta_data_samples.loc[meta_data_samples.PGDP_ID == i]["Species"].values[0] == "ursinus (grayfoot)":
        pop_list.append("ursinus")
    else:
        pop_list.append(meta_data_samples.loc[meta_data_samples.PGDP_ID == i]["Species"].values[0])
    inclusion_list.append(1)

In [26]:
pop_df = pd.DataFrame()
pop_df["ID"] = individuals
pop_df["population"] = pop_list
pop_df["inclusion"] = inclusion_list

In [28]:
pop_df.to_csv("/faststorage/project/baboondiversity/data/haploidified_chrX_males/idfile.txt",
              sep=" ", header=False, index=False)